In [1]:
import torch
print("Pytorch " + torch.__version__)
import pandas as pd
import numpy as np

from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import Trainer, TrainingArguments, EvalPrediction
from datasets import load_dataset

from sklearn import metrics
from scipy import stats
from functools import partial
from tqdm.auto import tqdm

from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Create device agnostic code
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

c:\Users\Dart_ilder\miniconda3\envs\.conda_Cas\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pytorch 1.13.1


'cuda'

In [2]:
cas_voc = pd.read_csv("./cas_voc.tsv", sep="\t")
cas_voc.head(3)

,Loci_id,Strand,Genome_assembly,Chromosome,N1,Protein_id,B2,Gene_id,Gene_family,Type,Species,Start,End,Seq,Prot,Embs
0,RN99_05230,-,GCA_001296125.1,CP012714.1,991.0,ALF19893_1,+,cd09644,csn2,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1100111,1100773,TTAGAAAATTTCACATAAATCATTATCTATAACAATTAAATTATCA...,MTFQYKGFNFKIDFEEKNIFSLIVENKRAYRKIIEDLVNNSNIEDG...,"5.101668462157249451e-02,4.367627203464508057e..."
1,RN99_05235,-,GCA_001296125.1,CP012714.1,992.0,ALF20727_1,+,mkCas0206,cas2,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1100770,1101075,TCATAAAACCACAAGCCTTTCATCTGTTTCTAAAAATGTCCCTTTT...,MRMLLFFDLPSVTNSDLKEYRKFRKFLIENGFSMLQESVYSKLLLH...,"6.999336928129196167e-02,-9.389735758304595947..."
2,RN99_05240,-,GCA_001296125.1,CP012714.1,993.0,ALF19894_1,+,cd09720,cas1,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1101080,1101958,CTATAACTCATCTTGAAAAAATCTCACTAATGATAAATCATTTGAG...,MSGWRVIIVTGRGKLDLRYNSISIRRDNGTDFIHIGEVNTLILETT...,"4.088897258043289185e-02,-6.158418953418731689..."


In [4]:
embs = cas_voc["Embs"]
labels = cas_voc["Gene_family"]
embs = [np.fromstring(row, sep=',') for row in embs]
len(embs)

36970

In [5]:
embs = np.array(embs)


X_train, X_test, y_train, y_test = train_test_split(embs, labels,
                                                    train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,
                      config_dict='TPOT MDR', warm_start=True, use_dask=True, periodic_checkpoint_folder="./tpot_checkpoints/")
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline.py')

Optimization Progress:  22%|██▎       | 27/120 [15:43<1:17:36, 50.07s/pipeline]